This notebook will extract eQTLs from the GTEx v.8 dataset, after you should intersect the generated file with your dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from collections import defaultdict
import glob
import os

import numpy as np
import pandas as pd


Download and process the [GTEx v8 data](https://storage.googleapis.com/adult-gtex/bulk-qtl/v8/single-tissue-cis-qtl/GTEx_Analysis_v8_eQTL.tar) and [GENCODE transcript .gtf file](https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_26/gencode.v26.annotation.gtf.gz) used in the GTEx analyses. Set the data filepaths accordingly:

In [ ]:
GTEX_DATA_DIR = '/content/drive/MyDrive/BimaProject/Modig1/Test3/GTEx_Analysis_v8_eQTL'
GENCODE_GTF = '/content/drive/MyDrive/BimaProject/Modig1/Test3/gencode.v26.annotation.gtf'
SEI_DIR = '/content/drive/MyDrive/BimaProject'

In [ ]:
ensg_gene_coordinates = {}
with open(GENCODE_GTF, 'r') as fh:
    for line in fh:
        if '##' in line:
            continue
        cols = line.strip().split('\t')
        coordinates = (cols[0], int(cols[3]), int(cols[4]))
        ensg_id = cols[8].split(';')[0].split(' ')[1][1:-1]
        ensg_gene_coordinates[ensg_id] = coordinates

Collect eQTLs for a tissue +/- `N_FROM_TSS` of any gene.

In [ ]:
N_FROM_TSS = 10000
tissue_variants_near_tss = defaultdict(list)
for fp in glob.glob(os.path.join(GTEX_DATA_DIR, '*signif_variant_gene_pairs.txt.gz')):
    tissue = os.path.basename(fp).split('.')[0]
    print('Processing {0}'.format(tissue))
    df = pd.read_csv(fp, sep='\t')
    for v, g in zip(df['variant_id'].tolist(), df['gene_id'].tolist()):
        gchrom, gpos, _ = ensg_gene_coordinates[g]
        cols = v.split('_')
        chrom = cols[0]
        pos = int(cols[1])
        if pos >= gpos - N_FROM_TSS and pos <= gpos + N_FROM_TSS:
            record = {'chrom': chrom,
                      'pos': pos,
                      'ref': cols[2],
                      'alt': cols[3],
                      'gene': g}
            tissue_variants_near_tss[tissue].append(record)

# Save to TSV file
output_filepath = '/content/drive/MyDrive/BimaProject/Modig1/Test3/file3.tsv'  # Provide the desired output file path
with open(output_filepath, 'w') as file:
    # Write header
    file.write("chrom\tpos\tref\talt\tgene\n")

    # Write data
    for tissue, records in tissue_variants_near_tss.items():
        for record in records:
            file.write("\t".join([str(record[key]) for key in ['chrom', 'pos', 'ref', 'alt', 'gene']]) + "\n")

Processing Adipose_Subcutaneous
Processing Adrenal_Gland
Processing Adipose_Visceral_Omentum
Processing Artery_Aorta
Processing Brain_Caudate_basal_ganglia
Processing Artery_Coronary
Processing Brain_Amygdala
Processing Brain_Anterior_cingulate_cortex_BA24
Processing Brain_Cortex
Processing Brain_Cerebellar_Hemisphere
Processing Brain_Cerebellum
Processing Brain_Hippocampus
Processing Brain_Frontal_Cortex_BA9
Processing Brain_Nucleus_accumbens_basal_ganglia
Processing Brain_Spinal_cord_cervical_c-1
Processing Brain_Hypothalamus
Processing Brain_Substantia_nigra
Processing Brain_Putamen_basal_ganglia
Processing Cells_EBV-transformed_lymphocytes
Processing Breast_Mammary_Tissue
Processing Artery_Tibial
Processing Colon_Sigmoid
Processing Esophagus_Gastroesophageal_Junction
Processing Colon_Transverse
Processing Cells_Cultured_fibroblasts
Processing Kidney_Cortex
Processing Heart_Left_Ventricle
Processing Esophagus_Muscularis
Processing Liver
Processing Minor_Salivary_Gland
Processing Eso

In [ ]:
# Import and merge datasets
filtered_data1 = pd.read_excel('/content/drive/MyDrive/BimaProject/Modig1/Merged_data_exome_filtered.xlsx')

filtered_data2 = pd.read_excel('/content/drive/MyDrive/BimaProject/Modig1/Merged_data_endoseq_filtered.xlsx')

# Load the previously generated file.tsv
file_tsv_filepath = '/content/drive/MyDrive/BimaProject/Modig1/Test3/file3.tsv'  # Provide the path to your file.tsv
file_tsv_df = pd.read_csv(file_tsv_filepath, sep='\t')

# Concatenate the dataframes along the rows
sei_data = pd.concat([filtered_data1, filtered_data2], ignore_index=True)

# Rename columns in sei_data
sei_data.rename(columns={
    'Chr':'chrom',
    'Start': 'pos',
    'REF allele': 'ref',
    'ALT allele': 'alt'
}, inplace=True)

# Print row count before merging
print(f"Number of rows in sei_data_df: {len(sei_data)}")
print(f"Number of rows in file_tsv_df: {len(file_tsv_df)}")

#NEW
# Merge the two dataframes based on chromosome, position, reference allele, and alternate allele
merged_df = pd.merge(sei_data, file_tsv_df, on=['chrom', 'pos', 'ref', 'alt'], how='inner')

# Drop duplicates based on the specified columns
merged_df = merged_df.drop_duplicates(subset=['chrom', 'pos', 'ref', 'alt'])

# Print row count after merging per sample
sample_row_counts = merged_df['Sample'].value_counts()
print("Number of rows in merged_df per sample:")
print(sample_row_counts)
# Print row count after merging
print(f"Number of rows in merged_df: {len(merged_df)}")

# Save the matching variants to a new Excel file
output_matched_filepath = '/content/drive/MyDrive/BimaProject/Modig1/vcf/eqtl10kb_file2.xlsx'  # Provide the desired path for the output file
merged_df.to_excel(output_matched_filepath, index=False, columns=['chrom', 'pos', 'ref', 'alt', 'Gene','GeneID','gene', 'Type', 'BioType', 'Sample'])



Number of rows in sei_data_df: 11775
Number of rows in file_tsv_df: 6357158
Number of rows in merged_df per sample:
Sample
5E_vs_5N    191
3E_vs_3N    153
2E_vs_2N     27
2O_vs_2N     10
4O_vs_4N      6
4E_vs_4N      6
3B_vs_3C      4
8B_vs_8C      3
9B_vs_9C      3
3O_vs_3N      2
6O_vs_6N      2
2B_vs_2C      2
6B_vs_6C      2
9A_vs_9C      1
3A_vs_3C      1
5O_vs_5N      1
6E_vs_6N      1
2A_vs_2C      1
Name: count, dtype: int64
Number of rows in merged_df: 416
